# **DESARROLLO DE MEDICAMENTOS EMPLEANDO INTELIGENCIA ARTIFICIAL**

 GRADO EN INGENIERÍA DE LA SALUD - *Trabajo Fin de Grado* 

Realizado por: *Beatriz García Dueñas* 

Tutor: *Juan Antonio Nepomuceno Chamorro*

Departamento: Lenguajes y Sistemas Informáticos


## **INTRODUCCIÓN A DEEPCHEM**

DeepChem es una biblioteca dedicada al aprendizaje automático científico. Su objetivo es facilitar la fase de experimentación aportando herramientas para cargar conjuntos de datos científicos, procesarlos, transformarlos, dividirlos y, sobre todo, aprender de ellos. De esta manera, se puede aplicar DeepChem para obtener múltiples beneficios como el descubrimiento de nuevos fármacos, aplicaciones en ciencia de materiales, biología, etc.  


En general, las ténicas de Deep Learning empleadas para entrenar nuestro modelo siguen todas la misma estructura:
1. Seleccionar el conjunto de datos (dataset) con el que se entrenará el modelo (o crear un nuevo conjunto de datos). 
2. Crear el modelo.
3. Entrenar el modelo con los datos existentes.
4. Evaluar el modelo en un conjunto de prueba independiente, con el fin de ver cómo funciona.
5. Usar el modelo para hacer predicciones sobre nuevos datos.

Con la librería DeepChem, cada uno de estos pasos puede resolverse mediante unas cuantas líneas de código Python. 

Empleando el comando "help" podemos obtener un poco de información sobre la librería deepchem. Podemos observar los diferentes submódulos que contiene, así como la versión disponible.

In [ ]:
!pip install --pre deepchem
import deepchem as dc
help(dc)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Help on package deepchem:

NAME
    deepchem - Imports all submodules

PACKAGE CONTENTS
    data (package)
    dock (package)
    feat (package)
    hyper (package)
    metalearning (package)
    metrics (package)
    models (package)
    molnet (package)
    rl (package)
    splits (package)
    trans (package)
    utils (package)

VERSION
    2.6.2.dev

FILE
    /usr/local/lib/python3.7/dist-packages/deepchem/__init__.py




A continuación, resolveremos un ejemplo para introducir las distintas funciones que tiene DeepChem en el campo del aprendizaje automático.

El problema que resolveremos está relacionado con la predicción de la solubilidad de moléculas dadas sus fórmulas químicas. La solubilidad es una propiedad muy relevante en el desarrollo de un fármaco ya que, si no es lo suficientemente soluble, no podrá introducirse en el torrente sanguíneo del paciente para alcanzar la diana terapéutica

Para comenzar, cargamos el conjunto de datos de solubilidades escogido, el dataset Delaney de MoleculeNet. Es el mismo conjunto de datos empleado en el ejemplo anterior.

In [ ]:
tasks, datasets, transformers = dc.molnet.load_delaney(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets



Una vez contamos con el conjunto de datos cargado, el siguiente paso es crear un modelo. Emplearemos un modelo denominado “Red Convolucional Gráfica” o “GraphConv” en DeepChem. 


In [ ]:
model = dc.models.GraphConvModel(n_tasks=1, mode='regression', dropout=0.2)

Una vez tenemos el modelo creado y el conjunto de datos, es hora de entrenar el modelo. Empleando la función .fit y pasando como parámetros el conjunto de datos de entrenamiento e indicando cuántas épocas de entrenamiento debe realizar, conseguiremos entrenar nuestro modelo.

In [ ]:
model.fit(train_dataset, nb_epoch=100)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_14:0", shape=(346,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_13:0", shape=(346, 64), dtype=float32), dense_shape=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Cast_4:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_17:0", shape=(1012,), dtype=int32), values=Tensor("gradient_tape/private__graph_conv_keras_model/graph_pool_1/Reshape_16

0.10859066963195801

Para comprobar si hemos entrenado correctamente nuestro modelo, procedemos a evaluarlo con el conjunto de datos de prueba. Para ello, seleccionamos una métrica de evaluación y llamamos a la función evaluate(). 

In [ ]:
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))

Training set score: {'pearson_r2_score': 0.9221124854768656}
Test set score: {'pearson_r2_score': 0.674851406602976}


Por último, estudiamos la evaluación obtenida. Como podemos observar, el conjunto de entrenamiento tiene más puntuación que el conjunto de prueba. Esto se debe a que los modelos funcionan mejor con los datos con los que se ha entrenado que con el conjunto de prueba.

Para determinar si los resultados obtenidos son óptimos o no, seguiremos la siguiente relación. 
-	Un modelo que produjera datos totalmente aleatorios obtendría una puntuación de 0.
-	Un modelo que hiciera sus predicciones de forma perfecta obtendría una puntuación de 1. 
Nuestro modelo se puede decir que lo hace bien dentro de estos parámetros

Nuestro modelo se puede decir que lo hace bien dentro de estos parámetros.


Para finalizar, aplicaremos este modelo creado para predecir las solubilidades de 10 moléculas del conjunto de prueba. Para cada una de las muestras, imprimiremos su cadena SMILES correspondiente y la log(solubilidad) predicha por el modelo. Para comparar estas predicciones, imprimiremos también la log(solubilidad) real del conjunto de prueba.

In [ ]:
solubilities = model.predict_on_batch(test_dataset.X[:10])
for molecule, solubility, test_solubility in zip(test_dataset.ids, solubilities, test_dataset.y):
    print(solubility, test_solubility, molecule)

[-1.6194464] [-1.60114461] c1cc2ccc3cccc4ccc(c1)c2c34
[0.84191555] [0.20848251] Cc1cc(=O)[nH]c(=S)[nH]1
[-0.49037457] [-0.01602738] Oc1ccc(cc1)C2(OC(=O)c3ccccc23)c4ccc(O)cc4 
[-1.8136808] [-2.82191713] c1ccc2c(c1)cc3ccc4cccc5ccc2c3c45
[-1.1180662] [-0.52891635] C1=Cc2cccc3cccc1c23
[1.7327256] [1.10168349] CC1CO1
[0.14837226] [-0.88987406] CCN2c1ccccc1N(C)C(=S)c3cccnc23 
[-1.2369052] [-0.52649706] CC12CCC3C(CCc4cc(O)ccc34)C2CCC1=O
[-1.2595742] [-0.76358725] Cn2cc(c1ccccc1)c(=O)c(c2)c3cccc(c3)C(F)(F)F
[0.03477405] [-0.64020358] ClC(Cl)(Cl)C(NC=O)N1C=CN(C=C1)C(NC=O)C(Cl)(Cl)Cl 


Tras este breve ejemplo, conocemos ahora un poco más sobre la librería DeepChem y una de sus aplicaciones a conjuntos de datos de moléculas.